In [ ]:
!head -n10 Dataset/airline.csv

In [ ]:
!head -n5 Dataset/carriers.txt

In [ ]:
import findspark
import pyspark
import random

findspark.init()
sc = pyspark.SparkContext(appName="Text Processing")

In [ ]:
airline = sc.textFile("~/Dataset/airline.csv")

In [ ]:
carriers = sc.textFile("BDLC-Project-G01/Dataset/carriers.txt")

In [ ]:
carriers

In [ ]:
%time airline.cache()

In [ ]:
%time carriers.cache()

In [ ]:
carriers.count()

In [ ]:
sc.stop()